# Решение задачи "Анализ ЭКГ-сигналов для диагностики сердечных патологий" для конкурса AI challenge

Оглавление
- Исследование области задачи
- Обработка и анализ данных
- Тестирование различных моделей

## Исследование области задачи
- SCP-ECG - http://masters.donntu.ru/2008/kita/golovach/library/4_ref/pub.html
- стадии инфаркта

## Обработка и анализ данных

Локальные переменные

In [ ]:
import os
os.environ['DATA_DIR'] = './data' # ты указываешь путь к своей папке

Используемые модули

In [ ]:
# Для данных
import pandas as pd
import numpy as np
import json
import os

# Для плюшек
import sklearn as sk

# Для красоты
import seaborn as sns
from pprint import pprint
import typing

Полезные функции

In [ ]:
def get_hr(folder: str, hr_num: str) -> np.array:
    with open(f'{os.environ["DATA_DIR"]}/{folder}/{hr_num}.npy', "rb") as f:
        return np.load(f, allow_pickle=True)


def flatten_list(lst: typing.List[any]) -> typing.List[any]:
    new_lst = []
    for elem in lst:
        if isinstance(elem, list):
            new_lst.extend(flatten_list(elem))
        else:
            new_lst.append(elem)
            
    return new_lst

Пример использования

In [ ]:
a = get_hr(folder='train', hr_num='15857_hr')
sns.lineplot(data=a.flatten()[:1500])

ecg_columns = ['report', 'scp_codes', 'heart_axis', 'infarction_stadium1', 'infarction_stadium2', 'validated_by', 
'second_opinion', 'initial_autogenerated_report', 'validated_by_human', 'baseline drift', 'static_noise', 'burst_noise', 'electrodes_problems', 
'extra_beats']

Загрузка данных

In [ ]:
meta = pd.read_csv(f'{os.environ["DATA_DIR"]}/train/train_meta.csv')
diagnosis = pd.read_csv(f'{os.environ["DATA_DIR"]}/train/train_gts.csv')

In [ ]:
meta.head(3)

In [ ]:
diagnosis.head(3)

Просмотрим инофрмацию о нашем датасете </br>
https://physionet.org/content/ptb-xl/1.0.3/ - еще здесь нада

In [ ]:
meta.info()

Удалим те данные, которые не несут важной для нас информации

In [ ]:
useless_columns = ['ecg_id', 'patient_id', 'nurse', 'site', 'device', 'recording_date', 'filename_lr', 'filename_hr']
meta.drop(columns=useless_columns, inplace=True)

Создадим список тех столбцов, которые возможно не несут важной информации (мы проверим это при обучении)

In [ ]:
strange_columns = ['age', 'sex', 'pacemaker', 'group']

Рассмотрим те столбцы, в которых есть много пропусков

In [ ]:
for column in meta.columns:
    print(f'Column name: {column} {round(meta[column].notna().sum() / len(meta) * 100, 2)}%')
    print(meta[column].value_counts() if len(meta[column].unique()) < 14 else f'so much unique values\n{meta[column].describe()}')
    print()

На основе этих данных выделим список столбцов с множество недостающих значений

In [ ]:
empty_columns = ['height', 'weight', 'heart_axis']
meta.drop(columns=empty_columns, inplace=True)

Удалим те строки, в которых electrodes_problem

In [ ]:
meta.drop(meta[meta['electrodes_problems'].notna()].index, inplace=True)
meta.drop(columns=['electrodes_problems'], inplace=True)
meta.drop(1514, inplace=True) # Хех пока

In [ ]:
meta = meta.reset_index(drop=True)

In [ ]:
meta.info()

Заменим строки в названиях, так как они малину портят

In [ ]:
str_columns = ['infarction_stadium1', 'infarction_stadium2', 'burst_noise', 'pacemaker', 'extra_beats']

meta = pd.get_dummies(meta, columns=str_columns, dtype=bool)

Обработаем оствашиеся не числовые столбцы (кроме report и record_name)
- baseline_drift
- static_noise
- scp_codes

In [ ]:
noise_types = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

def str_to_noise(s: str) -> typing.List[str]:
    if not isinstance(s, str):
        return []
    noises = []
    
    if 'alles' in s: # alles - all
        return noise_types
    
    vnable = False
    for elem in s.split(','):
        elem = elem.strip(',- ').upper()
        if not elem:
            continue
        if 'V' in elem and 'A' not in elem:
            vnable = True
        else:
            if vnable and elem in map(str, range(1, 7)):
                elem = 'V' + elem
        if '-' in elem:
            elem = elem.split('-')
            if 'V' in elem[0] and 'V' not in elem[1]:
                elem[1] = 'V' + elem[1]
            noises.extend(noise_types[noise_types.index(elem[0].strip(',- ')):noise_types.index(elem[1].strip(',- '))+1])
        else:
            noises.append(elem)
            
    return noises

baseline_drift

In [ ]:
for elem in noise_types:
    meta[f'baseline_drift_{elem}'] = elem in meta['baseline_drift'].apply(str_to_noise)

static_noise

In [ ]:
for elem in noise_types:
    meta[f'static_noise_{elem}'] = elem in meta['static_noise'].apply(str_to_noise)

scp_codes

In [ ]:
scp_uniq = meta['scp_codes'].unique().tolist()

In [ ]:
params = set(flatten_list([list(eval(codes).keys()) for codes in scp_uniq]))

In [ ]:
meta['scp_codes'] = meta['scp_codes'].apply(scp_convert)

for elem in params:
    meta[f'scp_{elem}'] = meta['scp_codes'].get(elem, None)
    print(meta[f'scp_{elem}'].unique())

In [ ]:
params

In [ ]:
meta['scp_RVH'].describe()

Пока пока

In [ ]:
meta.drop(columns=['baseline_drift', 'static_noise', 'extra_beats', 'scp_codes'])

In [ ]:
meta.info()

Отдельно рассмотрим столбец report, т.к. его нужно по особому обрабатывать

In [ ]:
reports = meta['report'].tolist()

In [ ]:
print(len(reports))
print(len(list(filter(lambda st: 'unbestÄtigter bericht' in st, reports)))) # неподтвержденные

In [ ]:
import stanza

ppln = stanza.Pipeline('de', processors='tokenize,pos,lemma')

In [ ]:
subrs = set()
prc = 0

for ind, report in enumerate(reports):
    doc = ppln(report)

    signs = ['!', ',', '.', ')', '(', ':']

    lemmas = []

    for sentence in doc.sentences:
        lemmas += [word.lemma if word.lemma not in signs else '' for word in sentence.words]

    
    subrs = subrs.union(set(filter(lambda x: x, lemmas)))
    if prc != round((ind+1)/len(reports)*100,1):
        prc = round((ind+1)/len(reports)*100, 1)
        print(prc)

In [ ]:
print(subrs, len(subrs))

TODO подробно описать каждый оставшийся столбец

In [ ]:
meta.info()

...

## Тестирование различных моделей

Используемые модули (не точно)

In [ ]:
# Нейронки (какие-то уберем, разобраться сначала, какие для этого подходят)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVR

# Посложнее, что успеем сделать
import torch #1
import keras #2

# Метрики
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score